<a href="https://colab.research.google.com/github/JoseMauricioBazagoitiaAyllon/Inteligencia-Artificial-II/blob/main/MLP1Se%C3%B1ales_trancito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os 
import torch
import torchvision
import torchvision.transforms as transforms

In [7]:
os.listdir("./drive/MyDrive/IA2/train")

['00000_00024.jpg',
 '00000_00023.jpg',
 '00000_00004 - copia - copia.jpg',
 '00000_00004 - copia (2).jpg',
 '00001',
 '00000']

In [8]:
train_dataset_path ="./drive/MyDrive/IA2/train"

In [9]:
training_transforms = transforms.Compose([
                                          transforms.Resize((30,30)),
                                          transforms.ToTensor()
])

In [10]:
train_dataset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform =training_transforms)

In [13]:
train_loader = torch.utils.data.DataLoader(dataset= train_dataset, batch_size = 32, shuffle= False)

In [19]:
def get_mean_and_std(loader):
  mean = 0.
  std = 0.
  total_image_count = 0
  for images,  _ in loader:
    image_count_in_a_batch = images.size(0)  
    print(images.shape)
    images = images.view(image_count_in_a_batch, images.size(1),-1)
    print(images.shape)
    mean += images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_image_count += image_count_in_a_batch
  mean/= total_image_count
  std /= total_image_count
  return mean, std

In [20]:
get_mean_and_std(train_loader)

torch.Size([20, 3, 30, 30])
torch.Size([20, 3, 900])


(tensor([0.5632, 0.4982, 0.5180]), tensor([0.1337, 0.1292, 0.1415]))

In [21]:
import torchvision
import torch
import torchvision.transforms as transforms
import os 
import matplotlib.pyplot as plt
import numpy as np

In [22]:
train_dataset_path ="./drive/MyDrive/IA2/train"
test_dataset_path ="./drive/MyDrive/IA2/test"

In [24]:
mean = [0.5632, 0.4982, 0.5180]
std = [0.1337, 0.1292, 0.1415]

train_transforms = transforms.Compose([
                                       transforms.Resize((30,30)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(10),
                                       transforms.ToTensor(),
                                       transforms.Normalize(torch.Tensor(mean),torch.tensor(std))            
])
test_transforms = transforms.Compose([
                                      transforms.Resize((30,30)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(torch.Tensor(mean),torch.tensor(std)) 
                                       
])

In [28]:
train_dataset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform =training_transforms) 
#test_dataset = torchvision.datasets.ImageFolder(root = test_dataset_path, transform =test_transforms) 

In [29]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 20
    Root location: ./drive/MyDrive/IA2/train
    StandardTransform
Transform: Compose(
               Resize(size=(30, 30), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [ ]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [ ]:
def crossentropy(output, target):
    logits = output[np.arange(len(output)),target]
    entropy = - logits + np.log(np.sum(np.exp(output),axis=-1))
    return entropy

def grad_crossentropy(output, target):
    answers = np.zeros_like(output)
    answers[np.arange(len(output)),target] = 1    
    return (- answers + softmax(output)) / output.shape[0]

In [ ]:
class Perceptron():
  def __init__(self, inputs, outputs):
    self.w = np.random.normal(loc=0.0, 
          scale = np.sqrt(2/(inputs+outputs)), 
          size = (inputs, outputs)) 
    self.ws = []
    self.loss = crossentropy
    self.grad_loss = grad_crossentropy
    
  def __call__(self, w, x):
    return np.dot(x, w) 

  def fit(self, x, y, epochs, lr, verbose=True, log_each=1):
    x = np.c_[np.ones(len(x)), x]
    for epoch in range(1,epochs+1):
        # Batch Gradient Descent
        y_hat = self(self.w, x)  
        # función de pérdida
        l = self.loss(y_hat, y).mean()
        # derivadas
        dldh = self.grad_loss(y_hat, y)
        dhdw = x
        dldw = np.dot(dhdw.T, dldh)
        # actualizar pesos
        self.w = self.w - lr*dldw
        # guardar pesos para animación
        self.ws.append(self.w.copy())
        # print loss
        if verbose and not epoch % log_each:
            print(f"Epoch {epoch}/{epochs} Loss {l}")
            
  def predict(self, x):
    x = np.c_[np.ones(len(x)), x]
    return self(self.w, x)

In [ ]:
perceptron = Perceptron(870, 2)
epochs, lr = 200, 1

In [ ]:
perceptron.fit(X_train, y_train, epochs, lr, log_each=10)

In [ ]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [ ]:
def linear(x):
    return x

def mse(y, y_hat):
    return 0.5*np.mean((y_hat - y.reshape(y_hat.shape))**2)

def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

In [ ]:
class MLP():
  def __init__(self, D_in, H, D_out):
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    self.loss = mse
    self.grad_loss = grad_mse

  def __call__(self, x):
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return y_hat
    
  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]        
            y_pred = self(x) 
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return y_hat

In [ ]:
model = MLP(D_in=1, H=3, D_out=1)
epochs, lr = 50, 0.0002
model.fit(data_train, 0, epochs, lr, batch_size=1, log_each=10)